In [1]:
from pyspark import SparkContext
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder\
	.config('spark.jars.packages', 'neo4j-contrib:neo4j-connector-apache-spark_2.12:4.0.1_for_spark_3')\
	.master("local").appName("spark_test").getOrCreate()

22/10/17 05:18:12 WARN Utils: Your hostname, irfandi resolves to a loopback address: 127.0.1.1; using 192.168.18.211 instead (on interface ens160)
22/10/17 05:18:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ddi/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ddi/.ivy2/cache
The jars for the packages stored in: /home/ddi/.ivy2/jars
neo4j-contrib#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-31f2735c-eba7-470d-8d8d-704702e85e8a;1.0
	confs: [default]
	found neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 in spark-packages
:: resolution report :: resolve 218ms :: artifacts dl 9ms
	:: modules in use:
	neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 from spark-packages in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: r

22/10/17 05:18:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/17 05:18:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


LOAD NODES 

CIF

In [12]:
# show data 
df = spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/cif.csv")
df.show(5)
print (df)


# df3.select("fullname").show()

+-----------+---------+--------------------+--------+---------+
|        cif|     name|             address|    city|     type|
+-----------+---------+--------------------+--------+---------+
|18346875234|     Layo|89012 Autumn Leaf...| Jakarta|WHOLESALE|
|18752032720| Topdrive|       83 Chive Pass|Surabaya|WHOLESALE|
|18806746420|    Meetz|9411 Schmedeman C...| Bandung|WHOLESALE|
|20476940876|     Vipe|4143 Daystar Terrace|  Bekasi|WHOLESALE|
|19168353628|Buzzshare|       6 Ohio Avenue|   Medan|WHOLESALE|
+-----------+---------+--------------------+--------+---------+
only showing top 5 rows

DataFrame[cif: string, name: string, address: string, city: string, type: string]


In [13]:
# LOAD DATA 
df.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("labels", ":Customer") \
  .option("node.keys", "cif") \
  .save()


22/10/17 05:36:19 WARN SchemaService: Switching to query schema resolution


ACCOUNT 

In [15]:
df1 = spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/account.csv")
df1.show(5)
print (df1)

+-----------+-----------+----------+
|        cif| account_no| open_date|
+-----------+-----------+----------+
|18346875234|  442222452|26-07-2017|
|18752032720|13335322450|21-12-2021|
|18752032720|13335322472|26-06-2013|
|18806746420| 2237072704|19-03-2021|
|18806746420|12015201770|16-07-2018|
+-----------+-----------+----------+
only showing top 5 rows

DataFrame[cif: string, account_no: string, open_date: string]


In [16]:
df1.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("labels", ":Account") \
  .option("node.keys", "account_no") \
  .save()


22/10/17 05:36:39 WARN SchemaService: Switching to query schema resolution


TRANSACTION

In [17]:
df2 = spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/transaction.csv")
df2.show(5)
print (df2)

+------+----------+------------+-----------+----------+-----------+
|trx_id|  trx_date|account_from|     amount|account_to|trx_channel|
+------+----------+------------+-----------+----------+-----------+
|  3482|22/06/2021|   599280088| 25031037.0|1755637562|        SMS|
|  4463|26/04/2021|    99677304|135827850.0|2266089634|        SMS|
|  9002|17/05/2021|    99677304| 45697816.0| 186714864|        SMS|
| 13021|26/08/2021|    18286974| 13895927.0| 349079568|        SMS|
| 13118|23/08/2021|   411650844|   106963.0| 458800460|        SMS|
+------+----------+------------+-----------+----------+-----------+
only showing top 5 rows

DataFrame[trx_id: string, trx_date: string, account_from: string, amount: string, account_to: string, trx_channel: string]


In [18]:
df2.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("labels", ":Transaction") \
  .option("node.keys", "trx_id") \
  .save()

22/10/17 05:38:53 WARN SchemaService: Switching to query schema resolution


RELATIONSHIP

HAS_ACCOUNT

In [19]:
df1 = spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/account.csv")
df1.show(5)
print (df1)

+-----------+-----------+----------+
|        cif| account_no| open_date|
+-----------+-----------+----------+
|18346875234|  442222452|26-07-2017|
|18752032720|13335322450|21-12-2021|
|18752032720|13335322472|26-06-2013|
|18806746420| 2237072704|19-03-2021|
|18806746420|12015201770|16-07-2018|
+-----------+-----------+----------+
only showing top 5 rows

DataFrame[cif: string, account_no: string, open_date: string]


In [ ]:
df1.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.type", "basic") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship", "HAS_ACCOUNT") \
  .option("relationship.source.save.mode", "Overwrite") \
  .option("relationship.source.labels", ":Customer") \
  .option("relationship.source.node.keys", "cif:cif") \
  .option("relationship.target.save.mode", "Overwrite") \
  .option("relationship.target.labels", ":Account") \
  .option("relationship.target.node.keys", "account_no:account_no") \
  .save()

SEND

In [24]:
df2 = spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/transaction.csv")
df2.show(5)
print (df2)

+------+----------+------------+-----------+----------+-----------+
|trx_id|  trx_date|account_from|     amount|account_to|trx_channel|
+------+----------+------------+-----------+----------+-----------+
|  3482|22/06/2021|   599280088| 25031037.0|1755637562|        SMS|
|  4463|26/04/2021|    99677304|135827850.0|2266089634|        SMS|
|  9002|17/05/2021|    99677304| 45697816.0| 186714864|        SMS|
| 13021|26/08/2021|    18286974| 13895927.0| 349079568|        SMS|
| 13118|23/08/2021|   411650844|   106963.0| 458800460|        SMS|
+------+----------+------------+-----------+----------+-----------+
only showing top 5 rows

DataFrame[trx_id: string, trx_date: string, account_from: string, amount: string, account_to: string, trx_channel: string]


In [27]:
df2.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.type", "basic") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship", "SEND") \
  .option("relationship.source.save.mode", "Overwrite") \
  .option("relationship.source.labels", ":Account") \
  .option("relationship.source.node.keys", "account_from:account_no") \
  .option("relationship.target.save.mode", "Overwrite") \
  .option("relationship.target.labels", ":Transaction") \
  .option("relationship.target.node.keys", "trx_id:trx_id") \
  .save()

22/10/17 05:51:59 WARN SchemaService: Switching to query schema resolution
22/10/17 05:51:59 WARN SchemaService: Switching to query schema resolution
22/10/17 05:51:59 WARN SchemaService: Switching to query schema resolution
22/10/17 05:51:59 WARN SchemaService: For the following exception
org.neo4j.driver.exceptions.ClientException: Unable to convert scala.collection.immutable.Map$Map1 to Neo4j Value.
	at org.neo4j.driver.Values.value(Values.java:134)
	at org.neo4j.driver.internal.util.Extract.mapOfValues(Extract.java:203)
	at org.neo4j.driver.internal.AbstractQueryRunner.parameters(AbstractQueryRunner.java:69)
	at org.neo4j.driver.internal.AbstractQueryRunner.run(AbstractQueryRunner.java:43)
	at org.neo4j.spark.service.SchemaService.retrieveSchemaFromApoc(SchemaService.scala:68)
	at org.neo4j.spark.service.SchemaService.liftedTree2$1(SchemaService.scala:171)
	at org.neo4j.spark.service.SchemaService.structForRelationship(SchemaService.scala:155)
	at org.neo4j.spark.service.SchemaServ

In [28]:
df4= spark.read.option("delimiter", ";").option("header", "true").csv("/home/ddi/spark-3.3.0-bin-hadoop3/data/bankingdata/transaction.csv")
df4.show(5)
print (df4)

+------+----------+------------+-----------+----------+-----------+
|trx_id|  trx_date|account_from|     amount|account_to|trx_channel|
+------+----------+------------+-----------+----------+-----------+
|  3482|22/06/2021|   599280088| 25031037.0|1755637562|        SMS|
|  4463|26/04/2021|    99677304|135827850.0|2266089634|        SMS|
|  9002|17/05/2021|    99677304| 45697816.0| 186714864|        SMS|
| 13021|26/08/2021|    18286974| 13895927.0| 349079568|        SMS|
| 13118|23/08/2021|   411650844|   106963.0| 458800460|        SMS|
+------+----------+------------+-----------+----------+-----------+
only showing top 5 rows

DataFrame[trx_id: string, trx_date: string, account_from: string, amount: string, account_to: string, trx_channel: string]


RECEIVE

In [29]:
df2.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.211:7687") \
  .option("database", "irfandi") \
  .mode("append") \
  .option("authentication.type", "basic") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship", "RECEIVE") \
  .option("relationship.source.save.mode", "Overwrite") \
  .option("relationship.source.labels", ":Transaction") \
  .option("relationship.source.node.keys", "trx_id:trx_id") \
  .option("relationship.target.save.mode", "Overwrite") \
  .option("relationship.target.labels", ":Account") \
  .option("relationship.target.node.keys", "account_to:account_no") \
  .save()

22/10/17 06:05:21 WARN SchemaService: Switching to query schema resolution
22/10/17 06:05:21 WARN SchemaService: Switching to query schema resolution
22/10/17 06:05:21 WARN SchemaService: Switching to query schema resolution
22/10/17 06:05:21 WARN SchemaService: For the following exception
org.neo4j.driver.exceptions.ClientException: Unable to convert scala.collection.immutable.Map$Map1 to Neo4j Value.
	at org.neo4j.driver.Values.value(Values.java:134)
	at org.neo4j.driver.internal.util.Extract.mapOfValues(Extract.java:203)
	at org.neo4j.driver.internal.AbstractQueryRunner.parameters(AbstractQueryRunner.java:69)
	at org.neo4j.driver.internal.AbstractQueryRunner.run(AbstractQueryRunner.java:43)
	at org.neo4j.spark.service.SchemaService.retrieveSchemaFromApoc(SchemaService.scala:68)
	at org.neo4j.spark.service.SchemaService.liftedTree2$1(SchemaService.scala:171)
	at org.neo4j.spark.service.SchemaService.structForRelationship(SchemaService.scala:155)
	at org.neo4j.spark.service.SchemaServ